# 1. NLP Exploration 2: Webscraping 
This exploration is an extension of **Domains Classifier V4**. We looked at just the domain name in that example, and utilized segmentation and a few other things in order to create a **vocabulary** of words that occur in the domain name. This vocabulary then one hot encoded in order to represent inputs, and then these inputs were fed into a **random forest** in order to try and map to the correct output class. 

We are now going to try and extended that by making use of **Webscraping**. Working with the same data set we will scrape the associated webpages and try to gather as much information as we can that may help us make predictions. Some things to note:
1. We will start by just looking at the home page. This clearly does not give us 100% of the information we may need, but it will be the first iteration.
2. We will also only be looking at visible text and not at css class names, commented out code, or anything else that would not be visible to a user. 
2. Based on the number of domains we have scores for (2000) we will need to be wary of how large we let our vocabulary size grow to. 

With that said, the first step is for us to actually scrape the home page of each domain. We can start by importing our necessary modules and loading in our data. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from util import (preprocess_analyst_scored_domains, class_imbalance)
%matplotlib inline

# Seaborn Plot Styling
sns.set(style="white", palette="husl")
sns.set_context("poster")
sns.set_style("ticks")

In [2]:
# Load our data
df = pd.read_csv('PPSDomains.csv')
df.head()

,domain,analystResult,BellonaStatus,medianLoadTime,speedPercentile,trafficDataRank,trafficDataReachRank,reachPerMillionValue,pageViewsPerMillionValue,pageViewsRankValue,usageStatisticRankValue,reachRankValue,historyDataScore,domainAge,registrantContactCountry,privateRegistrationStatus
0,07449m.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,68.0,UNITED STATES,"{""registrantName"":""Domain Administrator"",""simi..."
1,1037kissfm.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6955.0,UNITED STATES,NaN
2,1041kqth.com,NaN,TODO,7687.0,1.0,1941173.0,1872730.0,0.2,0.01,2322074.0,2025813.0,2060156.0,3,1251.0,UNITED STATES,NaN
3,1049maxcountry.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,978.0,UNITED STATES,NaN
4,1053thebuzz.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6221.0,UNITED STATES,NaN


In [3]:
# Call preprocess Utility Function
analystScoredDomains = preprocess_analyst_scored_domains(df)
analystScoredDomains.head()

,domain,analystResult,domainAge,registrantContactCountry,privateRegistrationStatus
10,10thousandcouples.com,0,-0.298104,0,0
11,1170kfaq.com,0,0.575157,1,0
12,11alive.com,0,1.205884,1,0
13,120gdyiyuan.com,0,-1.361280,1,0
14,12news.com,0,1.117373,1,0


---

<br>
## 1.1 Let the webscraping begin
Okay now that our data has been loaded and preprocessed, we can begin the process of webscrapping. In order to do that we will make use of the `urllib` and `BeatifulSoup` libraries.

In [4]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request

Now, lets go through the process of extracting the home page content for one domain, and then create functions to apply that to all of our domains. 

In [5]:
# Request the particular webpage
html = urllib.request.urlopen('http://12news.com').read()

In [8]:
# Function to take HTML and return only the visible text
def text_from_html(body):
  soup = BeautifulSoup(body, 'html.parser')
  texts = soup.findAll(text=True)
  visible_texts = filter(tag_visible, texts)
  return u" ".join(t.strip() for t in visible_texts)

# Function that determines whether the text would be visible on the web page
def tag_visible(element):
  if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
      return False
  if isinstance(element, Comment):
      return False
  return True

In [7]:
print(text_from_html(html))

                   WATCH LIVE  On Air 3:55PM          73  Phoenix, AZ   Phoenix Weather Summary: 73 degrees         Menu                           KPNX Alert Center   Live Video  Breaking News  Weather  Delays/ Closings      Sponsored by            News     Local  Money  Entertainment  Nation Now  Health  Politics  Life  I-Team      Weather & Traffic     Current Weather  Radar  Maps  Hourly & 10-Day  Heat  Monsoon  Traffic  Driving Smart  Gasbuddy      Sports     Latest Sports  High School Sports  High School Football  Cardinals & NFL  Suns & NBA  Diamondbacks & MLB  ASU & College  Coyotes & NHL  Olympics      Shows     Arizona Midday  Sunday Square Off      Features     12 Who Care  Call 12  DIY  Everything Arizona  Food  Health Check 12  Magnify Money  Recipes  Spanish  Verify  Ways 2 Save  Wednesday's Child  Your Voice      Media     Video  Photos      Connect     About Us  Contact Us  Advertise  Jobs  Meet the Team  Newsletters  TV Listings  YourTake              © 2018 KPNX-TV. Al